In [1]:
import pandas as pd
trad_flow = pd.read_csv('./RFM_TRAD_FLOW.csv', encoding='gbk')

In [5]:
trad_flow.head()

,transID,cumid,time,amount,type_label,type
0,9407,10001,14JUN09:17:58:34,199.0,正常,Normal
1,9625,10001,16JUN09:15:09:13,369.0,正常,Normal
2,11837,10001,01JUL09:14:50:36,369.0,正常,Normal
3,26629,10001,14DEC09:18:05:32,359.0,正常,Normal
4,30850,10001,12APR10:13:02:20,399.0,正常,Normal


### 通过RFM方法建立模型

#### 通过计算F反应客户对打折产品的偏好

In [33]:
trad_flow.groupby('type_label')['type_label'].count()

type_label
正常    15867
特价     1736
赠送     7989
退货     1070
Name: type_label, dtype: int64

In [34]:
trad_flow.groupby('type')['type_label'].count()

type
Normal            15867
Presented          7989
Special_offer      1736
returned_goods     1070
Name: type_label, dtype: int64

In [21]:
F = trad_flow.groupby(['cumid', 'type'])[['transID']].count()

In [22]:
F.head()

transID
cumid type                   
10001 Normal               15
      Presented             8
      Special_offer         2
      returned_goods        2
10002 Normal               12

In [23]:
F_trans = pd.pivot_table(F, index='cumid', columns='type', values='transID')
F_trans.head()

type,Normal,Presented,Special_offer,returned_goods
cumid,,,,
10001,15.0,8.0,2.0,2.0
10002,12.0,5.0,NaN,1.0
10003,15.0,8.0,1.0,1.0
10004,15.0,12.0,2.0,1.0
10005,8.0,5.0,NaN,1.0


数据空值检查

In [31]:
print(F_trans['Normal'].isnull().sum(), F_trans['Presented'].isnull().sum(), F_trans['Special_offer'].isnull().sum(),F_trans['returned_goods'].isnull().sum())

0 0 0 0


In [26]:
F_trans.fillna(0, inplace=True)

In [27]:
F_trans.head()

type,Normal,Presented,Special_offer,returned_goods
cumid,,,,
10001,15.0,8.0,2.0,2.0
10002,12.0,5.0,0.0,1.0
10003,15.0,8.0,1.0,1.0
10004,15.0,12.0,2.0,1.0
10005,8.0,5.0,0.0,1.0


根据打折的产品占打折产品和正常购买的占比，看兴趣度

In [35]:
F_trans['interest'] = F_trans['Special_offer']/(F_trans['Special_offer']+F_trans['Normal'])
F_trans.head()

type,Normal,Presented,Special_offer,returned_goods,interest
cumid,,,,,
10001,15.0,8.0,2.0,2.0,0.117647
10002,12.0,5.0,0.0,1.0,0.000000
10003,15.0,8.0,1.0,1.0,0.062500
10004,15.0,12.0,2.0,1.0,0.117647
10005,8.0,5.0,0.0,1.0,0.000000


#### 通过计算M反应客户的价值信息

In [36]:
M = trad_flow.groupby(['cumid', 'type'])[['amount']].sum()
M.head()

amount
cumid type                  
10001 Normal          3608.0
      Presented          0.0
      Special_offer    420.0
      returned_goods  -694.0
10002 Normal          1894.0

In [46]:
M_trans = pd.pivot_table(M, index='cumid', columns='type', values='amount')
M_trans['Special_offer'].fillna(0, inplace=True)
M_trans['returned_goods'].fillna(0, inplace=True)

In [47]:
M_trans['value'] = M_trans['Normal'] + M_trans['Special_offer'] + M_trans['returned_goods']
M_trans.head()

type,Normal,Presented,Special_offer,returned_goods,value
cumid,,,,,
10001,3608.0,0.0,420.0,-694.0,3334.0
10002,1894.0,0.0,0.0,-242.0,1652.0
10003,3503.0,0.0,156.0,-224.0,3435.0
10004,2979.0,0.0,373.0,-40.0,3312.0
10005,2368.0,0.0,0.0,-249.0,2119.0


#### 通过计算R反应客户是否活跃

In [79]:
from datetime import datetime
import time
trad_flow['time_new'] = trad_flow['time'].apply(lambda v: time.mktime(time.strptime(v, '%d%b%y:%H:%M:%S')))

In [80]:
trad_flow.head()

,transID,cumid,time,amount,type_label,type,time_new
0,9407,10001,14JUN09:17:58:34,199.0,正常,Normal,1.244974e+09
1,9625,10001,16JUN09:15:09:13,369.0,正常,Normal,1.245136e+09
2,11837,10001,01JUL09:14:50:36,369.0,正常,Normal,1.246431e+09
3,26629,10001,14DEC09:18:05:32,359.0,正常,Normal,1.260785e+09
4,30850,10001,12APR10:13:02:20,399.0,正常,Normal,1.271049e+09


In [82]:
R = trad_flow.groupby(['cumid'])[['time_new']].max()
R.head()

,time_new
cumid,
10001,1.284699e+09
10002,1.278129e+09
10003,1.282983e+09
10004,1.283057e+09
10005,1.282127e+09


#### 构建模型，筛选目标客户

In [66]:
from sklearn import preprocessing
threshold = pd.qcut(F_trans['interest'], 2, retbins=True)[1][1]

In [70]:
F_trans['interest'].values.reshape(-1, 1)

array([[0.11764706],
       [0.        ],
       [0.0625    ],
       ...,
       [0.28571429],
       [0.05882353],
       [0.09090909]])

分成有兴趣为1和没兴趣为0

In [71]:
binarizer = preprocessing.Binarizer(threshold=threshold)
interest_q = pd.DataFrame(binarizer.transform(F_trans['interest'].values.reshape(-1, 1)))

In [72]:
interest_q

,0
0,1.0
1,0.0
2,0.0
3,1.0
4,0.0
...,...
1195,0.0
1196,1.0
1197,1.0
1198,0.0


In [87]:
interest_q.index = F_trans.index
interest_q.columns = ['interest']

In [88]:
interest_q.head()

,interest
cumid,
10001,1.0
10002,0.0
10003,0.0
10004,1.0
10005,0.0


In [ ]:
有价值为1，无价值为0

In [76]:
threshold = pd.qcut(M_trans['value'], 2, retbins=True)[1][1]
binarizer = preprocessing.Binarizer(threshold=threshold)
value_q = pd.DataFrame(binarizer.transform(M_trans['value'].values.reshape(-1, 1)))
value_q.index=M_trans.index
value_q.columns=["value"]

In [83]:
threshold = pd.qcut(R["time_new"], 2, retbins=True)[1][1]
binarizer = preprocessing.Binarizer(threshold=threshold)
time_new_q = pd.DataFrame(binarizer.transform(R["time_new"].values.reshape(-1, 1)))
time_new_q.index=R.index
time_new_q.columns=["time"]

In [89]:
analysis = pd.concat([interest_q, value_q, time_new_q], axis=1)

In [90]:
analysis.head()

,interest,value,time
cumid,,,
10001,1.0,1.0,1.0
10002,0.0,0.0,0.0
10003,0.0,1.0,0.0
10004,1.0,1.0,0.0
10005,0.0,0.0,0.0


In [86]:
label = {
    (0,0,0):'无兴趣-低价值-沉默',
    (1,0,0):'有兴趣-低价值-沉默',
    (1,0,1):'有兴趣-低价值-活跃',
    (0,0,1):'无兴趣-低价值-活跃',
    (0,1,0):'无兴趣-高价值-沉默',
    (1,1,0):'有兴趣-高价值-沉默',
    (1,1,1):'有兴趣-高价值-活跃',
    (0,1,1):'无兴趣-高价值-活跃'
}

In [98]:
analysis

,interest,value,time,label
cumid,,,,
10001,1.0,1.0,1.0,有兴趣-高价值-活跃
10002,0.0,0.0,0.0,无兴趣-低价值-沉默
10003,0.0,1.0,0.0,无兴趣-高价值-沉默
10004,1.0,1.0,0.0,有兴趣-高价值-沉默
10005,0.0,0.0,0.0,无兴趣-低价值-沉默
...,...,...,...,...
40296,0.0,0.0,0.0,无兴趣-低价值-沉默
40297,1.0,1.0,0.0,有兴趣-高价值-沉默
40298,1.0,0.0,0.0,有兴趣-低价值-沉默


In [99]:
analysis['label'] = analysis.apply(lambda v:label[(v[0], v[1], v[2])], axis=1)

In [100]:
analysis.head()

,interest,value,time,label
cumid,,,,
10001,1.0,1.0,1.0,有兴趣-高价值-活跃
10002,0.0,0.0,0.0,无兴趣-低价值-沉默
10003,0.0,1.0,0.0,无兴趣-高价值-沉默
10004,1.0,1.0,0.0,有兴趣-高价值-沉默
10005,0.0,0.0,0.0,无兴趣-低价值-沉默
